In [118]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.offline import plot
import geocoder
import re
import json

In [251]:
inspection_data = pd.read_csv("inspections.csv")
inspection_data.head()

,web_certNumber,web_critical,web_customerNumber,web_direct,web_inspectionDate,web_legalName,web_nonCritical,web_reportLink,web_siteName,web_teachableMoments,...,pdf_customer_name,pdf_customer_addr,pdf_certificate,pdf_site_id,pdf_site_name,pdf_insp_type,pdf_date,pdf_animals_total,doccloud_url,licenseType
0,87-R-0002,0,2,0,2015-05-13,UTAH STATE UNIVERSITY,0,https://aphis--c.na107.content.force.com/sfc/d...,UTAH STATE UNIVERSITY,0,...,Utah State University,"V.P. For Research/1450 Old Main Hill\nLogan, U...",87-R-0002,001,UTAH STATE UNIVERSITY,ROUTINE INSPECTION,13-MAY-2015,214.0,https://www.documentcloud.org/documents/234700...,R
1,87-R-0002,0,2,0,2015-05-13,UTAH STATE UNIVERSITY,0,https://aphis--c.na107.content.force.com/sfc/d...,UTAH STATE UNIVERSITY,0,...,Utah State University,"V.P. For Research/1450 Old Main Hill\nLogan, U...",87-R-0002,002,UTAH STATE UNIVERSITY,ROUTINE INSPECTION,13-MAY-2015,0.0,https://www.documentcloud.org/documents/234700...,R
2,87-R-0002,0,2,0,2016-09-26,UTAH STATE UNIVERSITY,1,https://aphis--c.na107.content.force.com/sfc/d...,UTAH STATE UNIVERSITY,2,...,Utah State University,"V.P. For Research/1450 Old Main Hill\nLogan, U...",87-R-0002,001,UTAH STATE UNIVERSITY,ROUTINE INSPECTION,26-SEP-2016,653.0,https://www.documentcloud.org/documents/234700...,R
3,87-R-0002,0,2,0,2016-09-26,UTAH STATE UNIVERSITY,0,https://aphis--c.na107.content.force.com/sfc/d...,UTAH STATE UNIVERSITY,0,...,Utah State University,"V.P. For Research/1450 Old Main Hill\nLogan, U...",87-R-0002,002,UTAH STATE UNIVERSITY,ROUTINE INSPECTION,26-SEP-2016,0.0,https://www.documentcloud.org/documents/234701...,R
4,87-R-0002,0,2,0,2017-09-07,UTAH STATE UNIVERSITY,1,https://aphis--c.na107.content.force.com/sfc/d...,UTAH STATE UNIVERSITY,0,...,Utah State University,"V.P. For Research/1450 Old Main Hill\nLogan, U...",87-R-0002,001,UTAH STATE UNIVERSITY,ROUTINE INSPECTION,07-SEP-2017,594.0,https://www.documentcloud.org/documents/234701...,R


In [252]:
inspection_data.columns = [colName.lstrip('web_') for colName in inspection_data.columns]
inspection_data.columns = [colName.lstrip('pdf_') for colName in inspection_data.columns]

inspection_data.dtypes

certNumber           object
critical              int64
customerNumber        int64
irect                 int64
inspectionDate       object
legalName            object
nonCritical           int64
reportLink           object
siteName             object
teachableMoments      int64
hash_id              object
iscovered            object
insp_id              object
layout               object
customer_id         float64
customer_name        object
customer_addr        object
certificate          object
site_id              object
site_name            object
insp_type            object
ate                  object
animals_total       float64
occloud_url          object
licenseType          object
dtype: object

In [253]:
useful_columns = ['inspectionDate','insp_type','licenseType','critical','nonCritical','irect','teachableMoments','animals_total','legalName','siteName','customer_addr','reportLink']

In [254]:
inspection_data = inspection_data[useful_columns]
inspection_data.dtypes

inspectionDate       object
insp_type            object
licenseType          object
critical              int64
nonCritical           int64
irect                 int64
teachableMoments      int64
animals_total       float64
legalName            object
siteName             object
customer_addr        object
reportLink           object
dtype: object

In [255]:
#Critical and direct violations are both serious
#I think the word 'critical' gets this accross better, so I'll combine them into one column

inspection_data.loc[:,'critical'] = inspection_data['critical'] + inspection_data['irect']

In [257]:
inspection_data = inspection_data.drop('irect', axis=1)

In [102]:
# inspection_data['customer_addr'].str.split('\n', expand=True).sample(5)
def parse_address(df_column):
    # split_address = parsed['Address'].str.split(pat='\n',n=2,expand=True)
    parsed = df_column.str.extract(r'(?P<Street>[\w|\s|,|.]*)\n(?P<City>.*),\s(?P<State>[[A-Z]{2})\s(?P<ZIP>\d\d\d\d\d)',expand=True)
    # split_address = split_address.rename(columns={0:"Address1",1:"Address2",2:"Address3"})
    # parsed = split_address.join(parsed.drop(columns='Address'))
    parsed['Street'] = parsed['Street'].str.replace('\n',', ')

    return parsed

parse_address(inspection_data['customer_addr']).iloc[81671]

/home/gustav/Documents/Programming/Data_Analytics/Animal Welfare Inspections/myenv/lib/python3.10/site-packages/pandas/core/strings/accessor.py:2607: FutureWarning:

Possible nested set at position 52



Street    8520 Allison Pointe Blvd., Suite 400
City                              Indianapolis
State                                       IN
ZIP                                      46250
Name: 81671, dtype: object

In [259]:

multiple_criticals = inspection_data.query('critical > 1 | nonCritical > 8')
multiple_criticals = multiple_criticals.sort_values('animals_total',ascending = False, axis=0)
len(multiple_criticals)

660

In [260]:
# find any entries that have null data and review associated reports

def make_clickable(hyperlink):
    return f'<a target="_blank" href="{hyperlink}">{hyperlink}</a>'.lstrip()

multiple_criticals[multiple_criticals.isna().any(axis=1)].style.format({'reportLink':make_clickable})

,inspectionDate,insp_type,licenseType,critical,nonCritical,teachableMoments,animals_total,legalName,siteName,customer_addr,reportLink
64913,2014-05-25,ROUTINE INSPECTION,nan,0,16,0,39.000000,DEER FOREST EXOTIC ANIMAL SANCTUARY,DEER FOREST EXOTIC ANIMAL SANCTUARY,"6800 Indian Lane Coloma, MI 49038",https://aphis--c.na107.content.force.com/sfc/dist/version/download/?oid=00Dt0000000GyZH&ids=068t000000DnwIl&d=%2Fa%2F3d000000pfiR%2FpkTucwXX66bGmgn6PakHcdCNG5B57rwku95XkYJv8D0&asPdf=false
40371,2014-05-25,ROUTINE INSPECTION,nan,0,16,0,39.000000,DEER FOREST AMUSEMENTS INC,DEER FOREST AMUSEMENTS INC,"P O Box 817 Coloma, MI 49046",https://aphis--c.na107.content.force.com/sfc/dist/version/download/?oid=00Dt0000000GyZH&ids=068t000000DnwGG&d=%2Fa%2F3d000000pkPS%2FP5yIIPt6KPaZydB95Qyp1T1JZwzidJj2I3Eah4kMchY&asPdf=false
64914,2014-06-17,ROUTINE INSPECTION,nan,1,14,0,38.000000,DEER FOREST EXOTIC ANIMAL SANCTUARY,DEER FOREST EXOTIC ANIMAL SANCTUARY,"6800 Indian Lane Coloma, MI 49038",https://aphis--c.na107.content.force.com/sfc/dist/version/download/?oid=00Dt0000000GyZH&ids=068t000000Dnw20&d=%2Fa%2F3d000000pSaV%2FqsG481Uo1yHA885JgRuEn6LEdqDRYatwUSZhQWyDNpE&asPdf=false
40372,2014-06-17,ROUTINE INSPECTION,nan,1,14,0,38.000000,DEER FOREST AMUSEMENTS INC,DEER FOREST AMUSEMENTS INC,"P O Box 817 Coloma, MI 49046",https://aphis--c.na107.content.force.com/sfc/dist/version/download/?oid=00Dt0000000GyZH&ids=068t000000DnwJe&d=%2Fa%2F3d000000pkc6%2F.0cAif4b.OQzcptMOrkeHPmCOSsA1e0x0NMh65XN0TI&asPdf=false
18013,2023-01-11,nan,C,3,2,0,nan,City of El Paso,City of El Paso,nan,nan
29785,2023-01-05,nan,T,3,5,0,nan,KALITTA AIR LLC,KALITTA AIR LLC,nan,nan
38578,2016-08-09,ROUTINE INSPECTION,A,0,10,0,nan,NOLAN DEN BOER GINA DEN BOER,NOLAN AND GINA DEN BOER,"Nolan Den Boer 3271 Coolidge Ave Rock Valley, IA 51247",https://aphis--c.na107.content.force.com/sfc/dist/version/download/?oid=00Dt0000000GyZH&ids=068t000000DnwLp&d=%2Fa%2F3d000000pi5c%2Fev29uNj9Oxv24nxbuonWbICPfSTR8LPkD799tr7O688&asPdf=false


In [262]:
multiple_criticals['licenseType'].value_counts()

C    286
A    179
B    110
R     72
T      6
G      2
H      1
Name: licenseType, dtype: int64

In [263]:
# The 'City of El Paso' and 'Kalitta Air' entries have very limited information, so will be dropped
# The license type can easily be imputed from the report. In this case, they refer to an exotic animal sanctuary that was exhibiting without a license
# Class A refers to a breeder, B to a dealer, R to research, and C to an exhibitor
# Because it helps categorize them, we'll replace license type here with a C

# Reading the report on Nolan and Gina Den Boer reveals that they own dog breeding kennels, which the report indicates contain at least 16 animals, so that value will be updated

try:
    multiple_criticals = multiple_criticals.drop([18013,29785])
except:
    pass
multiple_criticals.at[38578,'animals_total'] = 16.0
multiple_criticals.at[64913,'licenseType'] = 'C'
multiple_criticals.at[40371,'licenseType'] = 'C'
multiple_criticals.at[40372,'licenseType'] = 'C'
multiple_criticals.at[64914,'licenseType'] = 'C'

In [295]:
# Also we'll replace the license type letters with the type of animal handler to make it clearer, except for G. 

type_dict = {
    'A': 'Breeder',
    'B': 'Dealer',
    'C': 'Exhibitor',
    'R': 'Research',
    'T': 'Carrier',
    'H': 'Intermediate Handler',
    'G': 'Unknown'
}

multiple_criticals['type'] = multiple_criticals['licenseType'].map(type_dict).astype('category')

In [265]:
#geotag the location using a call to ARCGIS geocoder

multiple_criticals['loc'] = multiple_criticals['customer_addr'].apply(geocoder.arcgis)

In [266]:
# This is what a location looks like, a JSON object with lots of information enabling data mapping
multiple_criticals['loc'].iloc[20].json

{'address': '30 N 3rd St, Womelsdorf, Pennsylvania, 19567',
 'bbox': {'northeast': [40.36643800978242, -76.18903596538213],
  'southwest': [40.364438009782425, -76.19103596538214]},
 'confidence': 9,
 'lat': 40.36543800978242,
 'lng': -76.19003596538214,
 'ok': True,
 'quality': 'PointAddress',
 'raw': {'name': '30 N 3rd St, Womelsdorf, Pennsylvania, 19567',
  'extent': {'xmin': -76.19103596538214,
   'ymin': 40.364438009782425,
   'xmax': -76.18903596538213,
   'ymax': 40.36643800978242},
  'feature': {'geometry': {'x': -76.19003596538214, 'y': 40.36543800978242},
   'attributes': {'Score': 100, 'Addr_Type': 'PointAddress'}}},
 'score': 100,
 'status': 'OK'}

In [267]:
# See the locations that have no confident geotag and seed if we can retag them using a different method
multiple_criticals[multiple_criticals['loc'].apply(lambda x: x.confidence < 8)]



,inspectionDate,insp_type,licenseType,critical,nonCritical,teachableMoments,animals_total,legalName,siteName,customer_addr,reportLink,type,loc
70574,2017-07-11,ROUTINE INSPECTION,G,2,2,3,24602.0,U S MEAT ANIMAL RESEARCH CENTER,US MEAT ANIMAL RESEARCH CENTER,"P.O. Box 166\nClay Center, NE 68933",https://aphis--c.na107.content.force.com/sfc/d...,???,"[[68933, Clay Center, Nebraska]]"
24524,2017-07-31,ROUTINE INSPECTION,A,2,2,0,4000.0,Oakwood Research Facility Inc,OAKWOOD RESEARCH FACILITY INC,"P.O. Box 455\nOxford, MI 48371",https://aphis--c.na107.content.force.com/sfc/d...,Breeder,"[[48371, Oxford, Michigan]]"
5881,2022-07-22,ROUTINE INSPECTION,R,2,0,0,2392.0,University of Puerto Rico,SABANA SECA,PO Box 365067 - Medical Science Campus\nSan Ju...,https://aphis--c.na107.content.force.com/sfc/d...,Research,[[00936]]
7760,2021-12-07,ROUTINE INSPECTION,R,2,0,0,1880.0,Bethyl Laboratories Inc.,"BETHYL LABORATORIES, INC.","P. O. BOX 850\nMONTGOMERY, TX 77356",https://aphis--c.na107.content.force.com/sfc/d...,Research,"[[77356, Montgomery, Texas]]"
6845,2022-05-16,ROUTINE INSPECTION,R,2,0,0,1441.0,Colorado State University,COLORADO STATE UNIVERSITY,203 Administration 2001 Campus Delivery\nFORT ...,https://aphis--c.na107.content.force.com/sfc/d...,Research,"[[80523, Fort Collins, Colorado]]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
27452,2021-08-24,FOCUSED INSPECTION,C,4,0,0,3.0,Lions Tigers & Bears Inc,"LIONS, TIGERS & BEARS, INC","P O BOX 2220\nARCADIA, FL 34265",https://aphis--c.na107.content.force.com/sfc/d...,Exhibitor,"[[34265, Arcadia, Florida]]"
15137,2016-06-21,ROUTINE INSPECTION,C,2,2,0,1.0,Zoologico De Puerto Rico Dr Juan A Rivero,ZOOLOGICAL DE P.R. AT MAYAGUEZ,"Po Box 9023207\nSan Juan, PR 00902",https://aphis--c.na107.content.force.com/sfc/d...,Exhibitor,[[00902]]
6650,2014-04-09,ROUTINE INSPECTION,R,2,5,0,0.0,ALLERGAN,ALLERGAN,"P.O. Box 19534\nIrvine, CA 92623",https://aphis--c.na107.content.force.com/sfc/d...,Research,"[[92623, Irvine, California]]"
5966,2017-04-04,FOCUSED INSPECTION,R,2,0,0,0.0,University of Washington,UNIVERSITY OF WASHINGTON,"Box 357160\nSeattle, WA 98195",https://aphis--c.na107.content.force.com/sfc/d...,Research,"[[98195, Seattle, Washington]]"


In [268]:
#They're mostly PO Boxes, which can be located to the City, State level at least, but our geocoder should already have done that for us.
#There are a couple of universities in there, but locating them to the ZIP code level should also be fine.
# We'll extract the latitude and longitude data
multiple_criticals['lat'] = multiple_criticals['loc'].apply(lambda x: x.lat)
multiple_criticals['lng'] = multiple_criticals['loc'].apply(lambda x: x.lng)


In [269]:
# The names of the sites are also bothering me. Some are upper case, some lower case, I'm going to make them all Camel Case for readability

multiple_criticals['siteName'] = multiple_criticals['siteName'].str.upper().str.title()
multiple_criticals['legalName'] = multiple_criticals['legalName'].str.upper().str.title()


In [296]:
# I should have everything I need to map these locations. I'm going to checkpoint my notebook here so I don't have to keep making API calls
# I'll export the small dataset I'm using as well. The full location information is stored as a ARCGIS object, so it won't export cleanly.
# I really only need the JSON data from it, so I'll extract that, then export the data

multiple_criticals['loc_json'] = multiple_criticals['loc'].apply(lambda x: x.json)
multiple_criticals.to_csv('multiple_criticals.csv') 

AttributeError: 'str' object has no attribute 'json'

In [271]:
# Starting from here when I reload the notebook
multiple_criticals = pd.read_csv('multiple_criticals.csv')



In [272]:
# Several reports are for the same location. Ideally I'll display them on the map in a way that prevents overlap
# I want to allow users to access each report individually by clicking

# Add a column that includes the number of reports for each location
multiple_criticals['reportCount']=multiple_criticals.groupby('lat')['lng'].transform('count')

# Write a function to slightly alter the latitude and longitude for overlapping points, so that they'll show up in a cluster

def jitter(x,jitter=0.00005):
    rand = np.random.randint(-5,5)
    return x + jitter * rand

# Copy lat and long columns
multiple_criticals['jitterLat'] = multiple_criticals.apply(lambda row: jitter(row['lat']) if row['reportCount'] > 1 else row['lat'], axis=1)
multiple_criticals['jitterLng'] = multiple_criticals.apply(lambda row: jitter(row['lng']) if row['reportCount'] > 1 else row['lng'], axis=1)


In [297]:
# Quick sanity check, are they all in the US? Are they plotted correctly? I'll create the plot structure I'm ultimately going to use to create the final map

# fig1 = go.FigureWidget([go.scattergeo(multiple_criticals, title='Animal Welfare Inspections 2014-2022', lat='lat', lon='lng', scope='north america', color='type', size = 'animals_total', size_max=30, hover_name= "siteName", hover_data=hover_data, fitbounds='locations', projection='natural earth', width=1000, height=800
# )])

fig = go.Figure(data=go.Scattergeo(
    lat=multiple_criticals['jitterLat'],
    lon=multiple_criticals['jitterLng'],
    text=multiple_criticals['legalName'],
    mode='markers',
    customdata=np.stack((multiple_criticals['critical'], multiple_criticals['nonCritical'], multiple_criticals['type'], multiple_criticals['reportLink']), axis=-1)
)

)

fig.show()

They're all there! Now to fine-tune the map. 

* The area should include just the United States - the map will be specified as a geo object
* Marker size depends on # of animals total (logarithmic scale)
* Marker color depends on whether there were critical violations or not
* There should be some jitter to allow users to access multiple reports from the same site
* On hover, each marker should show the name of the site and the number of critical and noncritical violations
* On click, each marker should open a new tab with the PDF report
* The legend will explain the types of site

In [298]:
multiple_criticals['has_criticals'] = multiple_criticals['critical'].apply(lambda x: 1 if x>0 else 0).astype('category')

In [299]:
# Start with the map

title = dict(text= ("U.S. Animal Welfare Inspections 2014-2022<br>" +
"<sup>All reports that found <span style='color:crimson'>2+ critical</span> issues or <span style='color:blue'>8+ non-critical</span> ones</sup>"),x=0.5, y=0.9, xanchor='center')
geo = dict(
    scope ='usa',
    projection_type='albers usa',
    showland = True,
    landcolor = 'rgb(230,230,230)',
    subunitwidth = 0.5,
    
)

source = "Source: USDA Animal and Plant Inspepction Service<br>(data compiled by Data Liberation Project)"

fig.update_layout(title=title,
                  title_font=dict(family='Open Sans',size=20),
                  geo=geo,
                  autosize=True)

fig.add_annotation(
    showarrow = False,
    text=source,
    font=dict(size=10,family='Open Sans'),
    xref='paper',
    xanchor='center',
    yref='paper',
    yanchor='auto',
    x=0.5, 
    y=-0.1
)

In [300]:
# Then the markers

marker = dict(
    size = multiple_criticals['animals_total'],
    sizeref = 60,
    sizemin = 3,
    sizemode = 'area',
    opacity = 0.8,
    symbol = 'circle',
    colorscale = ['blue','crimson'],
    color = multiple_criticals['has_criticals'].cat.codes
)
fig.update_traces(overwrite=True,marker=marker)

In [301]:
# Now the hovertext

hovertemplate = ("<b>%{text}</b><br>" + 
"Criticals: %{customdata[0]}<br>" +
"Non-criticals: %{customdata[1]}" +
"<extra>%{customdata[2]}</extra>")

fig.update_traces(overwrite=True, hovertemplate=hovertemplate)

In [302]:
# Add click functionality

data = fig.data

# Get HTML representation of plotly.js and this figure
plot_div = plot(fig, output_type='div', include_plotlyjs=True)

# Get id of html div element that looks like
# <div id="301d22ab-bfba-4621-8f5d-dc4fd855bb33" ... >
res = re.search('<div id="([^"]*)"', plot_div)
div_id = res.groups()[0]

# Build JavaScript callback for handling clicks
# and opening the URL in the trace's customdata 
js_callback = """
<script>
var plot_element = document.getElementById("{div_id}");
plot_element.on('plotly_click', function(data){{
    console.log(data);
    var point = data.points[0];
    if (point) {{
        console.log(point.customdata[3]);
        window.open(point.customdata[3], '_blank');
    }}
}})
</script>
""".format(div_id=div_id)

# Build HTML string
html_str = """
<html>
<head>
<meta content="application/pdf">
<title>Animal Welfare Inspections</title>
</head>
<body>
{plot_div}
{js_callback}
</body>
</html>
""".format(plot_div=plot_div, js_callback=js_callback)

with open('plot.html', 'w+') as f:
    f.write(html_str)